In [1]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
from IPython.display import display, HTML, Markdown
import math
import copy
from functools import reduce

import util
from library import render_table, render_multi_index_2D_table, drop_columns

## The cells below solves question 3a

Calculate the saturation pressure for a crude oil sample from Assignment 2 for the following feeds 

|Feed|CH₄ (mol%)|C₃H₈ (mol%)|Heavy oil(mol%)|T °C|Pb KPa|Swelling factor|Viscosity at Pb (cp)|
|----|----|----|----|----|----|----|----|
|#1|0|57.94|42.06|20.7|664.9|1.304|31.6|
|#2|13.81|49.52|36.67|20.5|4368.5|1.362|0|

In [2]:
# Working data
feeds = {
    "feed_1": {
        "sys_info": { "temp": util.to_rankine(20.7, 'c'), "Pbm": util.to_psi(664.9, 'kpa'), "swell": 1.304, "visco": 31.6 },
        "mixture": [
            {"component": "C\u2083", "name": "Propane", "xj": 0.5794, "tc": util.to_rankine(206.06, 'f'), "pc": 616.0, "w": 0.1522, "Mw": 49.097,},
            {"component": "H_O", "name": "HeavyOil", "xj": 0.4206, "tc": util.to_rankine(903.2, 'k'), "pc": util.to_psi(15.917, 'atm'),"w": 0.8537, "Mw": 389,},
        ],
        "bip_matrix": [
            [0,0], # propane
            [0,0]  # HO
        ]
    },
    "sample": {
        "sys_info": { "temp": util.to_rankine(100, 'f'), "Pbm": "", "swell": "", "visco": "" },
        "mixture": [
            { "component": "C\u2081", "name": "Methane", "xj": 0.7, "pc": 667.2, "tc": 343, "w": 0.008 },
            { "component": "n-C\u2085", "name": "n-Pentane", "xj": 0.3, "pc": 489.4, "tc": 845, "w": 0.251 }
        ],
        "bip_matrix": [
            [0, 0.021],
            [0.021, 0]
        ]
    }
}

In [3]:
# widgets definitions
pressure_input = widgets.FloatText(description="psia (ass.)", value="100")
feed_options = [(value, index) for index, value in enumerate(feeds.keys())]
feed_dd = widgets.Dropdown(options=feed_options, value=0, description="Feeds")

current_feed = {}
current_feed = feeds[list(feeds.keys())[feed_dd.value]]

In [4]:
def step_1_k_calc(mixture, pressure, temp):
    mixture_copy = mixture[:]
    #     usng wilsons correlation
    def calc_k_for_each(pci, tci, wi):
        term_1 = pci / pressure
        term_2 = (1 + wi)
        term_3 = (1 - (tci / temp))
        return term_1 * math.exp(5.37 * term_2 * term_3)
    
    for comp in mixture_copy:
        comp['k'] = calc_k_for_each(comp['pc'], comp['tc'], comp['w'])
        comp['yj'] = comp['k'] * comp['xj']
        
    return mixture_copy

In [5]:
# Calculate each component PR-EOS Properties
def step_2_eos_props(mixture, temp, pressure):
    mixture_copy = mixture[:]
    
    for comp in mixture_copy:
        pc = comp['pc']
        tc = comp['tc']
        w = comp['w']
        tr = round(util.calculate_tr_no_convert(temp, tc), 5)
        comp['tr'] = tr
        b = round(util.calculate_b_no_convert(tc, pc), 4)
        comp['b'] = b
        ac = round(util.calculate_ac_no_convert(tc, pc), 0)
        comp['ac'] = ac
        alpha = round(util.calculate_alpha_no_convert(w, temp, tc), 4)
        comp['alpha'] = alpha
        aT = round(util.calculate_at_no_convert(ac, alpha), 0)
        comp['aT'] = aT
        
    return mixture_copy

In [6]:
def step_3_phase_calculation(mixture, feed_dd_value, pressure, temp):
    mixture_copy = mixture[:]
    phase_data = { "liquid": {}, "gas": {} }
    
    aTjs = util.keys_to_list_of_dict('aT', mixture_copy)
    aTjs_list = list(aTjs.values())
    bjs = util.keys_to_list_of_dict('b', mixture_copy)
    bjs_list = list(bjs.values())
    current_bip = feeds[list(feeds.keys())[feed_dd_value]]['bip_matrix']
    
    # for liquid phase
    xjs = util.keys_to_list_of_dict('xj', mixture_copy)
    xjs_list = list(xjs.values())
    aT_liquid = util.calculate_aT_per_phase(xjs_list, aTjs_list, current_bip)
    b_liquid = util.calcuate_b_per_phase(xjs_list, bjs_list)
    A_liquid = util.calculate_A_no_convert(aT_liquid, pressure, temp)
    B_liquid = util.calculate_B_no_convert(b_liquid, pressure, temp )
    liquid = { 'aT': aT_liquid, 'b': b_liquid, "A": A_liquid, "B": B_liquid }
    phase_data['liquid'] = liquid
    
    # for gas phase
    yjs = util.keys_to_list_of_dict('yj', mixture_copy)
    yjs_list = list(yjs.values())
    aT_gas = util.calculate_aT_per_phase(yjs_list, aTjs_list, current_bip)
    b_gas = util.calcuate_b_per_phase(yjs_list, bjs_list)
    A_gas = util.calculate_A_no_convert(aT_gas, pressure, temp)
    B_gas = util.calculate_B_no_convert(b_gas, pressure, temp )
    gas = { 'aT': aT_gas, 'b': b_gas, "A": A_gas, "B": B_gas }
    phase_data['gas'] = gas
    
    return phase_data

In [7]:
def step_4_z_factor(phase_data):
    phase_data_copy = phase_data.copy()
    
    liquid_phase_roots = util.cubic_root_calculation_for_PR_EOS(phase_data_copy['liquid']['A'], phase_data_copy['liquid']['B'], True)
    liquid_real_roots = util.filter_out_complex_numbers(liquid_phase_roots)
    z_liquid = util.get_highest_or_lowest_root(liquid_real_roots, 'liquid')
    phase_data_copy['liquid']['z'] = z_liquid
    
    gas_phase_roots = util.cubic_root_calculation_for_PR_EOS(phase_data_copy['gas']['A'], phase_data_copy['gas']['B'], True)
    gas_real_roots = util.filter_out_complex_numbers(gas_phase_roots)
    z_gas = util.get_highest_or_lowest_root(gas_real_roots, 'gas')
    phase_data_copy['gas']['z'] = z_gas
    
    return phase_data_copy

In [8]:
def step_5_composition_coef(comp_list, phase_data, feed_dd_value):
    current_bip = feeds[list(feeds.keys())[feed_dd_value]]['bip_matrix']
    comp_list_copy = copy.deepcopy(comp_list)
    
    def calculate_summation(target_comp, phase, bip_to_use):
        """The phase can either be xj for liquid and yj for gas"""
        sum = 0
        for ind, value in enumerate(comp_list_copy):
            sum += value[phase] * ((target_comp['ac'] * value['ac'] * target_comp['alpha'] * value['alpha']) ** 0.5) * (1 - bip_to_use[ind])
        return sum
    
    for index, comp in enumerate(comp_list_copy):
        liquid_Bj = comp['b'] / phase_data['liquid']['b']
        liquid_Aj = (2 * calculate_summation(comp, 'xj', current_bip[index])) / phase_data['liquid']['aT']
        comp['liquid'] = {'Aj': liquid_Aj, 'Bj': liquid_Bj}
        
        gas_Bj = comp['b'] / phase_data['gas']['b']
        gas_Aj = (2 * calculate_summation(comp, 'yj', current_bip[index])) / phase_data['gas']['aT']
        comp['gas'] = {'Aj': gas_Aj, 'Bj': gas_Bj}
    
    new_list = [{'component': d['component'], 'liquid': d['liquid'], 'gas': d['gas']} for d in comp_list_copy]
    return new_list

In [9]:
def step_6_fugacity_coef(component_coef_data, original_data, phase_data):
    merged = [dict1 | dict2 for dict1 in component_coef_data for dict2 in original_data if dict1['component'] == dict2['component']]
    
    def solve_fug_coef(comp, phase):
        term_1 = comp[phase]['Bj'] * (phase_data[phase]['z'] - 1)
        term_2 = math.log(phase_data[phase]['z'] - phase_data[phase]['B'])
        term_3 = (phase_data[phase]['A'] / ((2**1.5) * phase_data[phase]['B']))
        term_4 = (comp[phase]['Aj'] - comp[phase]['Bj'])
        numerator = phase_data[phase]['z'] + (((2 ** 0.5) + 1) * phase_data[phase]['B'])
        denom = phase_data[phase]['z'] - (((2 ** 0.5) - 1) * phase_data[phase]['B'])
        term_5 = math.log(numerator / denom)
        
        result = math.exp(term_1 - term_2 - ( term_3 * term_4 * term_5 ))
        return result
    fug_coef_data = []
    for component in merged:
        fug_coef_l = solve_fug_coef(component, 'liquid')
        fug_coef_g = solve_fug_coef(component, 'gas')
        
        fug_coef_data.append({"component": component['component'], "fug_coef_l": fug_coef_l, "fug_coef_g": fug_coef_g})
    return fug_coef_data

In [10]:
def step_7_fugacity_k_and_error(fug_coef_data, original_data, pressure):
    merged = [dict1 | dict2 for dict1 in fug_coef_data for dict2 in original_data if dict1['component'] == dict2['component']]
    
    fug_data = []
    err_sum = 0
    new_pressure = 0
    for comp in merged:
        fug_l = comp['fug_coef_l'] * (comp['xj'] * pressure)
        fug_g = comp['fug_coef_g'] * (comp['yj'] * pressure)
        k_new = comp['fug_coef_l'] / comp['fug_coef_g']
        yj_new = comp['xj'] * k_new
        err = (1 - (fug_l / fug_g)) ** 2
        err_sum += err
        new_pressure += fug_l / comp['fug_coef_g']
        
        
        fug_data.append({
            "component": comp['component'],
            "xj": comp['xj'],
            "yj": comp['yj'],
            "fug_coef_l": comp['fug_coef_l'], 
            "fug_coef_g": comp['fug_coef_g'],
            "fug_l": fug_l,
            "fug_g": fug_g,
            "ε": err,
            "k_new": k_new,
            "yj_new": yj_new,
        })
    return {"data": fug_data, "error": err_sum, "new_pressure": round(new_pressure, 1) }
        

In [11]:
def on_any_change(feed_dd_value, pressure):
    current_feed = feeds[list(feeds.keys())[feed_dd_value]]
    mixture = copy.deepcopy(current_feed['mixture'])
    temp = current_feed['sys_info']['temp']
    display_tables = [0,1,2,3,4,5,6,7,8,9]
    
    display_tables[0] = pd.DataFrame.from_dict([current_feed['sys_info']])
    display_tables[1] = pd.DataFrame(mixture)
    
    # The first time to get k_values via wilson
    k_mixture = step_1_k_calc(mixture, pressure, temp)
    display_tables[2] = pd.DataFrame(k_mixture)
    
    loop_pressure = pressure
    error = 1000000000000000
    number_of_loops = 0
    while error > (10**-10):
        number_of_loops += 1
        loop_initial_k_mixture = k_mixture[:]
        
        eos_props_mixture = step_2_eos_props(loop_initial_k_mixture, temp, loop_pressure)
        display_tables[3] = pd.DataFrame(eos_props_mixture)

        phase_data = step_3_phase_calculation(eos_props_mixture, feed_dd_value, loop_pressure, temp)
        display_tables[4] = pd.DataFrame(phase_data).transpose()

        phase_data_with_z = step_4_z_factor(phase_data)
        display_tables[5] = pd.DataFrame(phase_data_with_z).transpose()

        component_coef_data = step_5_composition_coef(eos_props_mixture, phase_data, feed_dd_value)
        component_coef_data_df = render_multi_index_2D_table(component_coef_data, 'liquid', 'gas', 'Aj', 'Bj', 'component', False)
        display_tables[6] = component_coef_data_df

        fugacity_coef_data = step_6_fugacity_coef(component_coef_data, eos_props_mixture, phase_data_with_z)
        display_tables[7] = pd.DataFrame(fugacity_coef_data)

        fugacity_data = step_7_fugacity_k_and_error(fugacity_coef_data, eos_props_mixture, loop_pressure)
        display_tables[8] = pd.DataFrame(fugacity_data['data'])
        display_tables[9] = pd.DataFrame({"New_Pressure": [fugacity_data['new_pressure']]}).transpose()

        # update for the loop
        error = fugacity_data['error']
        loop_pressure = fugacity_data['new_pressure']
        
        # K_mixture is the mixture from after wilson
        for comp in loop_initial_k_mixture:
            for new_comp in fugacity_data['data']:
                if comp['component'] == new_comp['component']:
                    comp['k'] = new_comp['k_new']
                    comp['yj'] = new_comp['yj_new']

    for each_df in display_tables:
        display(each_df)
    print(f"Loops: {number_of_loops}")
interactive_widgets = widgets.interactive(
    on_any_change,
    feed_dd_value=feed_dd,
    pressure=pressure_input
)
display(interactive_widgets)

interactive(children=(Dropdown(description='Feeds', options=(('feed_1', 0), ('sample', 1)), value=0), FloatTex…